In [1]:
from selenium import webdriver
from urllib import request
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import re
import sys
import json

In [2]:
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
# """
# 起動1
# java -jar C:\Users\mmi-lab\Documents\selenium\selenium-server-standalone-3.141.59.jar -role hub

# 起動2
# java -Dwebdriver.chrome.driver="C:\Users\mmi-lab\Documents\selenium\chromedriver.exe" -jar C:\Users\mmi-lab\Documents\selenium\selenium-server-standalone-3.141.59.jar -role node -hub http://192.168.1.224:4444

# """

# """
# 使えそう
# ジャンルからフレーズ・例文を探す
# https://gogakuru.com/english/phrase/genre/index.html

# """

In [25]:
class Crawler:
    def __init__(self, ) -> None:
        self.phrases = []
    
    def connect(self, server=None):
        chrome_options = webdriver.ChromeOptions()
        if server:
            command = "http://{0}:4444/wd/hub".format(server)
            self.driver = webdriver.Remote(
                command_executor="http://{0}:4444/wd/hub".format(server),
                options=chrome_options
            )
    
    def _in(self, search, target):
        """
        search : list or str
        target : str
        """
        result = True
        if isinstance(search, list):
            for s in search:
                result = result and s in target
            return result
        elif isinstance(search, str):
            return search in target
        else:
            print("list or str is expected, but not")
            return False
    
    # def _extract_stop_type(self, target):
    #     if self.essential not in target:
    #         return None
    #     for stop in self.stop_list:
    #         if stop in target:
    #             return stop
    #     return None
    
    # def _extract_label(self, text):
    #     if self.mode == "YN":
    #         return ["YN", self._extract_stop_type(text)]
    #     # 後で
    #     elif self.mode == "WH":
    #         return ["WH", self._extract_stop_type(text)]
    #     elif self.mode == "plain":
    #         return ["plain"]
    #     elif self.mode == "proposal":
    #         return ["proposal", self._extract_stop_type(text)]
    #     else:
    #         return ["please"]
    
    def _go_conv_page(self):
        element = self.driver.find_element_by_class_name("condition").find_elements_by_tag_name("a")
        element[1].click()
        
    def crawl(self):


        self.driver.get("https://gogakuru.com/english/phrase/genre/index.html")
        
        self._go_conv_page()

        html = self.driver.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'lxml')
        while 1:
            cells = soup.find_all("td", class_="summary")
            for cell in cells:
                # engs = cell.find_all("span", class_="font-en")
                # text_ = [eng.get_text() for eng in engs]
                # text = text_[0]
                # # print(text)
                # # 大文字で含まれるならば
                # stop = self._extract_stop_type(text)
                # if  stop:
                jp = cell.find("dd", class_="jp").get_text()
                self.phrases.append(jp)
                    # self.label.append( self._extract_label(text) )
                    # print("stop:{0}, jp:{1}".format(stop, jp))

            # 一度打ち切り
            if len(self.phrases) % 50 == 0:
                print("the size of crawled phrase is {0}".format(len(self.phrases)))
        
            # 次のurl
            try:
                next_t = self.driver.find_element_by_class_name("right").find_element_by_tag_name('a')
                next_t.click()
            except:
                break
            html = self.driver.page_source.encode('utf-8')
            soup = BeautifulSoup(html, 'lxml')
        
        

    # def make_corpus(self):
    #     self.corpus = {}
    #     self.corpus[self.mode] = []
    #     for t in zip(self.phrases):
    #         one_data = {"data":t}
    #         self.corpus[self.mode].append(one_data)
    
    def write(self, filename):
        df = pd.DataFrame(self.phrases, columns=["phrase"] )
        df.to_csv(filename, index=False)
    
    def quit(self):
        self.driver.quit() 



In [26]:
data_path = "../../corpus/gogakuru/"
# name = "question/short"
# name = "question"
# name = "case_
# data_path = "/".join([corpus_root, name]) + "/"
# data_path
dataM = DataManager(data_path)
import os
print(os.listdir(data_path))

['test.csv']


In [27]:
crawler = Crawler()
crawler.connect("192.168.1.224")

In [28]:
crawler.crawl()

the size of crawled phrase is 100
the size of crawled phrase is 200
the size of crawled phrase is 300
the size of crawled phrase is 400
the size of crawled phrase is 500
the size of crawled phrase is 600
the size of crawled phrase is 700
the size of crawled phrase is 800
the size of crawled phrase is 900
the size of crawled phrase is 1000
the size of crawled phrase is 1100
the size of crawled phrase is 1200
the size of crawled phrase is 1300
the size of crawled phrase is 1400
the size of crawled phrase is 1500
the size of crawled phrase is 1600
the size of crawled phrase is 1700
the size of crawled phrase is 1800
the size of crawled phrase is 1900
the size of crawled phrase is 2000
the size of crawled phrase is 2100
the size of crawled phrase is 2200
the size of crawled phrase is 2300
the size of crawled phrase is 2400
the size of crawled phrase is 2500
the size of crawled phrase is 2600
the size of crawled phrase is 2700
the size of crawled phrase is 2800
the size of crawled phrase is

In [29]:
corpus_name = "test.csv"
crawler.write(data_path+corpus_name)

In [30]:
crawler.phrases
df = pd.DataFrame(crawler.phrases, columns=["phrase"] )
df.to_csv(data_path+corpus_name, index=False)